# Packages Section

In [0]:
#Link to my google drive and change directory to current directory
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/machine learning folder/IT Project')

Mounted at /content/drive


In [0]:
 #importing manipulations of data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_score,f1_score,roc_curve,roc_auc_score


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Model Act manipulation
import torch
import torch as tch
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F
from collections import OrderedDict
from PIL import  Image
import helper

# Data Section

In [0]:
class Dataloader(Dataset):

  #initialize the dataloader instance
  def __init__(self, X,y,train=True):
    self.X, self.y = X,y

    if train:
      self.trans = transforms.Compose([ transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])])
    else:
      self.trans = transforms.Compose([ transforms.Resize((224,224)),
                                        transforms.ToTensor()])

    #one hotendcode target
    self.encod = OneHotEncoder(categories='auto',sparse=False)
    self.encod.fit(np.array([[0],[1]]))

  def __getitem__(self, idx):
    #check if last sample 
    if idx == (self.__len__()):
      raise StopIteration
      
    x_,y_ = self.X[idx], self.y[idx]
    trans = self.trans

    #open images and resize
    img = Image.open(x_[0])
    img = trans(img).numpy().transpose((1, 2, 0))

    img2 = Image.open(x_[1])
    img2 = trans(img2).numpy().transpose((1, 2, 0))
    
    #one hot encode target
    out = np.array([y_]) #output
    out = self.encod.transform(out)

    #concate images together as flatten
    con_img = np.expand_dims(np.c_[img,img2],0)
    img_d = np.copy(con_img.transpose((0, 3, 1, 2)))  

    img_d = tch.tensor(img_d.squeeze(0).astype('float32'))
    target = tch.tensor(out.squeeze(0).astype('float32'))

    
    return img_d, target

  def __len__(self):
    return len(self.X)


  #class method to initialize test and train class instances
  @classmethod
  def pre_pro_dset(cls, prt_fol_pth="Pictures"):

      dset = cls.fold_to_reldset(prt_fol_pth = prt_fol_pth)
      x_trn, y_trn, x_val, y_val, x_tst, y_tst = cls.shuffle_reldset(rel_dset=dset, split_d=True)

      return cls(x_trn, y_trn,train=True), cls(x_val, y_val,train=False), cls(x_tst, y_tst,train=False)

  #static method to create reference to the images 
  @staticmethod
  def fold_to_reldset(prt_fol_pth = None):
      prt_fol = os.listdir(prt_fol_pth)
      nw_dset = np.empty((0,3))

      #for each folder in the dataset
      for ech_samp in prt_fol:
        #for each picture in a folder
        for samp1 in os.listdir(os.path.join(prt_fol_pth, ech_samp)):
          #for pictures in a folder label as 1
          for samp2 in os.listdir(os.path.join(prt_fol_pth, ech_samp)): 
            
            #create reference to images
            img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
            img2 = np.array(os.path.join(prt_fol_pth, ech_samp, samp2))

            #concate reference to images together
            out = np.array([1]) #output
            con_img = np.c_[img,img2]
            img_d = np.copy(np.c_[con_img,out])

            #add img to dataset
            nw_dset = np.r_[nw_dset,img_d]   

            #over sample the dataset, by adding the same instance
            nw_dset = np.r_[nw_dset,img_d]   

          #for selecting different folder to label as 0
          for samp3 in prt_fol:
            #if current folder is selected ---> skip
            if samp3 == ech_samp:
              continue
            #if current folder is not the one initially selected
            else:
              #select pictures in the newly selected folder
              for samp4 in os.listdir(os.path.join(prt_fol_pth, samp3)):
                #open images and resize
                img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
                img2 = np.array(os.path.join(prt_fol_pth, samp3, samp4))

                #concate images together as flatten
                out = np.array([0]) #output
                con_img = np.c_[img,img2]
                img_d = np.copy(np.c_[con_img,out])

                #add img to dataset
                nw_dset = np.r_[nw_dset,img_d] 

      return nw_dset

  #static methos to shuffle the reference to the images and split then
  @staticmethod
  def shuffle_reldset(rel_dset=None,X=None,y=None,split_d=True,test_s=0.2):
    #splitting dataset into train,remaining
    np.random.seed(42)
    #try:
    if str(type(rel_dset)) == "<class 'numpy.ndarray'>":
      X = rel_dset[:,:-1]
      y = np.float32(rel_dset[:,-1:])
    
  #except:
    try:
        split = StratifiedShuffleSplit(test_size=test_s)
        for trn_idx, tst_idx in split.split(X,y):
            x_trn, x_tst = X[trn_idx], X[tst_idx]
            y_trn, y_tst = y[trn_idx], y[tst_idx]

        if split_d == True:
          split = StratifiedShuffleSplit(test_size=0.5)

          for trn_idx, tst_idx in split.split(x_tst,y_tst):
              x_val_, x_tst_ = X[trn_idx], X[tst_idx]
              y_val_, y_tst_ = y[trn_idx], y[tst_idx]

          return (x_trn, y_trn, x_val_, y_val_, x_tst_, y_tst_) 
        else:
          return (np.r_[x_trn,x_tst], np.r_[y_trn,y_tst])

    except:
      print('x and y is NONE')

# Train Section

In [0]:
#initialize the train and test data instance
train, val, test = Dataloader.pre_pro_dset()
train_loader = DataLoader( train, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val, batch_size=32, shuffle=True, num_workers=2)
tst_loader = DataLoader(test, batch_size=32, shuffle=True, num_workers=4)

In [0]:
#download model
squeeze = models.squeezenet1_1(pretrained=True)

model = squeeze

#change input to cnn
model.features[0] = nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2))

#change input to fc
model.classifier[1] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
                                                
model = nn.Sequential(OrderedDict([('model', model),
                                   ('softmax_out',nn.Softmax(dim=1))]))

#set parameters to require gradient
for param in model.parameters():
  if (ct >= 1 and ct <= 2) or (ct >= 50):
    param.requires_grad = True
  else:
    param.requires_grad = False
  ct += 1

In [0]:
squeeze = tch.load('model_gen1.pt')

model = squeeze['model_arc']
model.load_state_dict(squeeze['state_dict'])

ct = 1

for param in model.parameters():
  if (ct >= 1 and ct <= 2) or (ct >= 50):
    param.requires_grad = True
  else:
    param.requires_grad = False
  ct += 1

In [0]:
int(len(val_loader.dataset)/32)*32

3264

In [0]:
#loss and optimizer
criterion = nn.BCELoss()
optimizer1 = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [0]:
class train_model:
  
  def __init__(self,model = None):
    self.model = model
    self.criterion = None
    self.optimizer = None
    self.weights = None
    self.t_epch = 0
    self.loaded_tr = 0
    self.loaded_vl = 0
    self.train_loss    = []
    self.test_loss     = []
    self.train_acc     = []
    self.train_pre     = []
    self.train_rec     = []
    self.train_f1_scr  = []
    self.test_acc      = []
    self.test_pre      = []
    self.test_rec      = []
    self.test_f1_scr   = []
    
  def train(self,train_loader=None, valid_loader=None,criterion=None,optimizer=None,epochs=5,batch_size=[32,32],lst_val_scr=0):
    
    model = self.model
    valid_rec_min = lst_val_scr
    train_on_gpu = tch.cuda.is_available()
    self.t_epch += epochs

    if len(batch_size) == 2:
      batch_s_t = batch_size[0]
      batch_s_v = batch_size[1]
    else:
      batch_s_t = batch_size[0]
      batch_s_v = batch_size[0]    

    for epoch in range(epochs):
      # keep track of training and validation loss and scores

        train_loss    = 0.0
        test_loss     = 0.0
        train_acc     = 0.0
        train_pre     = 0.0
        train_rec     = 0.0
        train_f1_scr  = 0.0
        test_acc      = 0.0
        test_pre      = 0.0
        test_rec      = 0.0
        test_f1_scr   = 0.0
      
      ###################
      # train the model #
      ###################
        for data, target in train_loader:
            self.loaded_tr += len(data)
       
          # move tensors to GPU if CUDA is available
            if train_on_gpu: 
                model.cuda()
                model.train()
                data, target = data.cuda(), target.cuda()
            else:
                model.cpu()
                model.train()
                data, target = data.cpu(), target.cpu()
     
          # clear the gradients of all optimized variables
            optimizer.zero_grad()
     
            print(f'in Train-->({self.loaded_tr}) \n')          
          
          
          # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #print(f'output_train is {output} and target is {target}')
          # calculate the batch loss
            loss = criterion(output, target)                   
      
          # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
      
          # perform a single optimization step (parameter update)
            optimizer.step()
       
          # update training loss
            train_loss     +=  loss.item()
            train_acc      +=  accuracy_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_pre      +=  precision_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_rec      +=  recall_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_f1_scr   +=  f1_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            self.save_model()
        self.loaded_tr = 0


      ######################    
      # validate the model #
      ######################
        model.eval()
     
        for data, target in valid_loader:
            self.loaded_vl += len(data)

            # move tensors to GPU if CUDA is available
            if train_on_gpu: 
                model.cuda()
                model.train()
                data, target = data.cuda(), target.cuda()
            else:
                model.cpu()
                model.train()
                data, target = data.cpu(), target.cpu()

            print(f'in val-->({self.loaded_vl}) \n ')
                
          # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #print(f'output_val is {output} and target is {target}')
          # calculate the batch loss
            loss = criterion(output, target)
                    
          # update average validation loss 
            test_loss     += loss.item()
            test_acc       += accuracy_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_pre       += precision_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_rec       +=  recall_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_f1_scr    +=  f1_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            
        self.loaded_vl = 0      

      # calculate average losses and scores
        train_loss = train_loss/round(len(train_loader.dataset)/batch_s_t)
        test_loss = test_loss/round(len(valid_loader.dataset)/batch_s_v)
      
        train_acc       =   1 if  train_acc/round(len(train_loader.dataset)/batch_s_t) > 1 else train_acc/round(len(train_loader.dataset)/batch_s_t)

        train_pre       =   1 if train_pre/round(len(train_loader.dataset)/batch_s_t) > 1 else train_pre/round(len(train_loader.dataset)/batch_s_t)

        train_rec       =   1 if train_rec/round(len(train_loader.dataset)/batch_s_t) > 1 else train_rec/round(len(train_loader.dataset)/batch_s_t)
                                     
        train_f1_scr    =   1 if train_f1_scr/round(len(train_loader.dataset)/batch_s_t) > 1 else train_f1_scr/round(len(train_loader.dataset)/batch_s_t)

        test_acc        =   1 if test_acc/round(len(valid_loader.dataset)/batch_s_v) > 1 else test_acc/round(len(valid_loader.dataset)/batch_s_v)

        test_pre        =   1 if test_pre/round(len(valid_loader.dataset)/batch_s_v) > 1 else test_pre/round(len(valid_loader.dataset)/batch_s_v)
  
        test_rec        =   1 if test_rec/round(len(valid_loader.dataset)/batch_s_v,5) > 1 else test_rec/round(len(valid_loader.dataset)/batch_s_v,5)

        test_f1_scr    =   1 if test_f1_scr/round(len(valid_loader.dataset)/batch_s_v,5) > 1 else test_f1_scr/round(len(valid_loader.dataset)/batch_s_v,5) 

      

            
      #append train scores
        self.train_loss.append(train_loss)
        self.train_acc.append(train_acc)
        self.train_rec.append(train_rec)
        self.train_f1_scr.append(train_f1_scr)
        self.train_pre.append(train_pre)
        
      #append test scores
        self.test_loss.append(test_loss)
        self.test_acc.append(test_acc)
        self.test_rec.append(test_rec)
        self.test_f1_scr.append(test_f1_scr)
        self.test_pre.append(test_pre)
        
      # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} '.format(epoch, train_loss, test_loss))
      
        print('train Accuracy score : {:.6f} \ttrain Precision Score: {:.6f} \ttrain Recall Score: {:.6f}\ttrain f1 Score: {:.6f}'.format(train_acc, train_pre, train_rec, train_f1_scr))
      
        print('test Accuracy score : {:.6f} \ttest Precision Score: {:.6f} \ttest Recall Score: {:.6f}\ttest f1 Score: {:.6f}'.format(test_acc, test_pre, test_rec, test_f1_scr))

        # save model if validation loss has decreased
        if test_rec > valid_rec_min:
            print('test recall increased ({:.6f} --> {:.6f}).  Saving model ...\n\n'.format(valid_rec_min,test_rec))
            
            #Save State_dict
            model_info = {'model_arc'  : model,
                             'state_dict' : model.state_dict(),
                              'val_recall' : test_rec}
            

            torch.save(model_info, 'model_deploy1.pt')
            valid_rec_min = test_rec

  def predict(self,data):
    model = self.model
    model.cpu().eval()
    
    data = data.cpu()

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    
    return output.cpu().topk(1)[1].detach().numpy()
  
  
  def save_model(self):
    model = self.model
    
    #Save State_dict
    model_info = {'model_arc'  : model,
                      'state_dict' : model.state_dict()}

    torch.save(model_info, 'model_gen1.pt')

In [0]:
model = train_model(model=model)
model.train(train_loader=train_loader,valid_loader=val_loader,criterion=criterion,optimizer=optimizer1,epochs=5,batch_size=[32,32],lst_val_scr=0)

in Train-->(32) 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type SqueezeNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.p

in Train-->(64) 

in Train-->(96) 

in Train-->(128) 

in Train-->(160) 

in Train-->(192) 

in Train-->(224) 

in Train-->(256) 

in Train-->(288) 

in Train-->(320) 

in Train-->(352) 

in Train-->(384) 

in Train-->(416) 

in Train-->(448) 

in Train-->(480) 

in Train-->(512) 

in Train-->(544) 

in Train-->(576) 

in Train-->(608) 

in Train-->(640) 

in Train-->(672) 

in Train-->(704) 

in Train-->(736) 

in Train-->(768) 

in Train-->(800) 

in Train-->(832) 

in Train-->(864) 

in Train-->(896) 

in Train-->(928) 

in Train-->(960) 

in Train-->(992) 

in Train-->(1024) 

in Train-->(1056) 

in Train-->(1088) 

in Train-->(1120) 

in Train-->(1152) 

in Train-->(1184) 

in Train-->(1216) 

in Train-->(1248) 

in Train-->(1280) 

in Train-->(1312) 

in Train-->(1344) 

in Train-->(1376) 

in Train-->(1408) 

in Train-->(1440) 

in Train-->(1472) 

in Train-->(1504) 

in Train-->(1536) 

in Train-->(1568) 

in Train-->(1600) 

in Train-->(1632) 

in Train-->(1664) 

in Train-->(

In [0]:
#testing modelb
mod = tch.load('fcmodel_train.pt')
fc = mod['model_arc']
fc.load_state_dict(mod['state_dict'])

resnet = torch.load('cnnmodel_train.pt')
cnn = resnet['model_arc']
cnn.load_state_dict(resnet['state_dict'])

#model
model = train_model(cnn=cnn, model=fc)

#data
data_x = np.load('features.npy')
data_y  = np.load('targets.npy')

#data to tensor data
data_x = tch.tensor(data_x.astype(np.float32))
data_y = tch.tensor(data_y.astype(np.float32)) 

#predict accuracy
# accuracy_score(model.predict(data_x[:50,:,:,:]).numpy(),data_y[:50])

In [0]:
model.save_model()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Softmax. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

# reserved codes

In [0]:
# # img = np.copy(np.ones((1, 301056)))
# # img2 = np.copy(np.zeros((1, 301056)))
# img = (plt.imread('Pictures/fes/DSC_0269.JPG')/225 - [0.485, 0.456, 0.406])/[0.229, 0.224, 0.225]
# img2 = plt.imread('Pictures/blaq/IMG_20190428_004049.jpg')
# # np.c_[img,img2].shape
# plt.imshow(img)
# # img

In [0]:
# #function to load dataset
# def fold_to_datast(prt_fol_pth = 'Pictures'):
#   siz = 224
#   prt_fol = os.listdir(prt_fol_pth)
#   nw_dset = np.empty((0,224,224,6))
#   target = np.empty((0,1))
#   trans = transforms.Compose([ transforms.Resize((224,224)),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406],
#                                                            [0.229, 0.224, 0.225])])

#   #for each folder in the dataset
#   for ech_samp in prt_fol:

#     #for each picture in a folder
#     for samp1 in os.listdir(os.path.join(prt_fol_pth, ech_samp)):

#       #for pictures in a folder label as 1
#       for samp2 in os.listdir(os.path.join(prt_fol_pth, ech_samp)):
        
#         #open images and resize
#         img = Image.open(os.path.join(prt_fol_pth, ech_samp, samp1))
#         img = trans(img).numpy().transpose((1, 2, 0))

#         img2 = Image.open(os.path.join(prt_fol_pth, ech_samp, samp2))
#         img2 = trans(img2).numpy().transpose((1, 2, 0))
        
#         #concate images together as flatten
#         out = np.array([[1]]) #output
#         con_img = np.expand_dims(np.c_[img,img2],0)
# #         img_d = np.copy(np.c_[con_img,out])
#         img_d = np.copy(con_img)

#         #add img to dataset
# #         print(f'shape = {img_d.shape}')
#         nw_dset = np.r_[nw_dset,img_d]
#         target = np.r_[target,out]
                          
# #         print(f'{ech_samp+samp1} and {ech_samp+samp2} is {1}')

#       #for selecting different folder to label as 0
#       for samp3 in prt_fol:

#         #if current folder is selected ---> skip
#         if samp3 == ech_samp:
#           continue

#         #if current folder is not the one initially selected
#         else:
#           #select pictures in the newly selected folder
#           for samp4 in os.listdir(os.path.join(prt_fol_pth, samp3)):
            
#             #open images and resize
#             img = Image.open(os.path.join(prt_fol_pth, ech_samp, samp1))
#             img = trans(img).numpy().transpose((1, 2, 0))
            
#             img2 = Image.open(os.path.join(prt_fol_pth, samp3, samp4))
#             img2 = trans(img2).numpy().transpose((1, 2, 0))
             
#             #concate images together as flatten
#             out = np.array([[0]]) #output
#             con_img = np.expand_dims(np.c_[img,img2],0)
#             img_d = np.copy(con_img)
#             nw_dset = np.r_[nw_dset,img_d]
#             target = np.r_[target,out]
# #             print(f'{ech_samp+samp1} and {samp3+samp4} is {0}')

#   return nw_dset,target

In [0]:
# def fold_to_datast(prt_fol_pth = 'Pictures'):
#   prt_fol = os.listdir(prt_fol_pth)
#   nw_dset = np.empty((0,3))

#   #for each folder in the dataset
#   for ech_samp in prt_fol:
#     #for each picture in a folder
#     for samp1 in os.listdir(os.path.join(prt_fol_pth, ech_samp)):
#       #for pictures in a folder label as 1
#       for samp2 in os.listdir(os.path.join(prt_fol_pth, ech_samp)): 
        
#         #open images and resize
#         img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
#         img2 = np.array(os.path.join(prt_fol_pth, ech_samp, samp2))

#         #concate images together as flatten
#         out = np.array([1]) #output
#         con_img = np.c_[img,img2]
#         img_d = np.copy(np.c_[con_img,out])

#         #add img to dataset
#         nw_dset = np.r_[nw_dset,img_d]   

#         #over sample the dataset, by adding the same instance
#         nw_dset = np.r_[nw_dset,img_d]   

# #       #for selecting different folder to label as 0
#       for samp3 in prt_fol:
#         #if current folder is selected ---> skip
#         if samp3 == ech_samp:
#           continue
#         #if current folder is not the one initially selected
#         else:
#           #select pictures in the newly selected folder
#           for samp4 in os.listdir(os.path.join(prt_fol_pth, samp3)):
#             #open images and resize
#             img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
#             img2 = np.array(os.path.join(prt_fol_pth, samp3, samp4))

#             #concate images together as flatten
#             out = np.array([0]) #output
#             con_img = np.c_[img,img2]
#             img_d = np.copy(np.c_[con_img,out])

#             #add img to dataset
#             nw_dset = np.r_[nw_dset,img_d]  
#   print(f'dset : {nw_dset.shape}')
#   return nw_dset

In [0]:
# class pre_pro_dset:

#   def __init__(self, prt_fol_pth='Pictures'):
#       self.fold_to_reldset(prt_fol_pth = prt_fol_pth)
#       self.shuffle_reldset()

#   def fold_to_reldset(self,prt_fol_pth = None):
#       prt_fol = os.listdir(prt_fol_pth)
#       nw_dset = np.empty((0,3))

#       #for each folder in the dataset
#       for ech_samp in prt_fol:
#         #for each picture in a folder
#         for samp1 in os.listdir(os.path.join(prt_fol_pth, ech_samp)):
#           #for pictures in a folder label as 1
#           for samp2 in os.listdir(os.path.join(prt_fol_pth, ech_samp)): 
            
#             #open images and resize
#             img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
#             img2 = np.array(os.path.join(prt_fol_pth, ech_samp, samp2))

#             #concate images together as flatten
#             out = np.array([1]) #output
#             con_img = np.c_[img,img2]
#             img_d = np.copy(np.c_[con_img,out])

#             #add img to dataset
#             nw_dset = np.r_[nw_dset,img_d]   

#             #over sample the dataset, by adding the same instance
#             nw_dset = np.r_[nw_dset,img_d]   

#           #for selecting different folder to label as 0
#           for samp3 in prt_fol:
#             #if current folder is selected ---> skip
#             if samp3 == ech_samp:
#               continue
#             #if current folder is not the one initially selected
#             else:
#               #select pictures in the newly selected folder
#               for samp4 in os.listdir(os.path.join(prt_fol_pth, samp3)):
#                 #open images and resize
#                 img = np.array(os.path.join(prt_fol_pth, ech_samp, samp1))
#                 img2 = np.array(os.path.join(prt_fol_pth, samp3, samp4))

#                 #concate images together as flatten
#                 out = np.array([0]) #output
#                 con_img = np.c_[img,img2]
#                 img_d = np.copy(np.c_[con_img,out])

#                 #add img to dataset
#                 nw_dset = np.r_[nw_dset,img_d] 

#       self.rel_dset = nw_dset

#   def shuffle_reldset(self):
#     #splitting dataset into train,remaining
#     split = StratifiedShuffleSplit(test_size=0.2)
#     X = self.rel_dset[:,:-1]
#     y = np.float32(self.rel_dset[:,-1:])

#     for trn_idx, tst_idx in split.split(X,y):
#         x_trn, x_tst = X[trn_idx], X[tst_idx]
#         y_trn, y_tst = y[trn_idx], y[tst_idx]

#     self.train_X, self.train_y = (x_trn, y_trn)
#     self.test_X, self.test_y = (x_tst, y_tst)

In [0]:
#encod = OneHotEncoder(categories='auto')
#y_ = encod.fit_transform(y).toarray()

#splitting dataset into train,remaining
#split = StratifiedShuffleSplit(test_size=0.2)
#
#for trn_idx, tst_idx in split.split(X,y):
#    x_trn, x_tst = X[trn_idx], X[tst_idx]
#    y_trn, y_tst = y[trn_idx], y[tst_idx]

#encod = OneHotEncoder(categories='auto')
#y_trn= encod.fit_transform(y_trn).toarray()
#y_tst= encod.fit_transform(y_tst).toarray()

#train_target = tch.tensor(y_trn.astype(np.float32))
#train =        tch.tensor(x_trn.astype(np.float32)) 
#
#test_target =  tch.tensor(y_tst.astype(np.float32)) 
#test =         tch.tensor(x_tst.astype(np.float32))

#batch_size = 6
#train_tensor = tch.utils.data.TensorDataset(train, train_target) 
#train_loader = tch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
#
#val_tensor =  tch.utils.data.TensorDataset(test, test_target) 
#val_loader =  tch.utils.data.DataLoader(dataset = val_tensor, batch_size = batch_size, shuffle = True)

In [0]:
#load the images with their references
  def load_paths(self, X, y):
    self.loaded += len(X)
    siz = 224
    #prt_fol = os.listdir(prt_fol_pth)
    nw_dset = np.empty((0,224,224,6))
    target = np.empty((0,1))
    trans = transforms.Compose([ transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

    
    for x_,y_ in zip(X,y):
        #open images and resize
        img = Image.open(x_[0])
        img = trans(img).numpy().transpose((1, 2, 0))

        img2 = Image.open(x_[1])
        img2 = trans(img2).numpy().transpose((1, 2, 0))
        
        #one hot encode target
        out = np.array([y_]) #output

        #concate images together as flatten
        con_img = np.expand_dims(np.c_[img,img2],0)
        img_d = np.copy(con_img)

        #add img to dataset
        nw_dset = np.r_[nw_dset,img_d]
        target = np.r_[target,out]    
    target = encod.transform(target)
    return tch.tensor(nw_dset.astype(np.float32)), tch.tensor(target.astype(np.float32))


# reset the state of the dataloader and shuffle the image references
  def reset(self):

    #set len of dataset and the loaded dataset
    self.len_relset = 0
    self.loaded = 0

    #shuttle the dataset
    self.X, self.y = self.shuffle_reldset(X=self.X,y=self.y,split_d=False, test_s=0.5) 

  def __loaded__(self):
    return self.loaded


  # iteration method
  def __iter__(self):
    return self 

  #get next image
  def __next__(self):
    self.len_relset += self.batch_size
    
    if ((self.len_relset-1)== self.X.shape[0]):
      self.reset()
      raise StopIteration
    return self.load_paths(self.X[self.len_relset:self.len_relset+self.batch_size], self.y[self.len_relset:self.len_relset+self.batch_size])

In [0]:
model.predict(data_x[100:150,:,:,:])

In [0]:
2.21848991e-02

In [0]:
data_y[100:150]

In [0]:
#Testing the model
features = tch.tensor(X.astype(np.float32))
targets = tch.tensor(y.astype(np.float32)) 

features[0:10,:,:,:].shape
#loading cnn
# resnet = torch.load('trmodel.pt')
# cnn = resnet['model_arc']
# cnn.load_state_dict(resnet['state_dict'])

# #loading neural net
# nw_nt = torch.load('itmodel1.pt')
# neural_network = nw_nt['model_arc']
# neural_network.load_state_dict(nw_nt['state_dict'])

# #Using class
# Final_model = train_model(cnn=cnn, model=neural_network)

In [0]:
# np.expand_dims(np.c_[img,img2],0).shape

In [0]:
# trans = transforms.Compose([ transforms.Resize((224,224)),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406],
#                                                            [0.229, 0.224, 0.225])])
# img = Image.open('Pictures/wonder/DSC_0261.JPG')
# img2 = Image.open('Pictures/blaq/IMG_20190428_235929.jpg')
# img = trans(img)
# img2 = trans(img2)
# plt.imshow(np.c_[img,img2][:,:,:224].reshape(224,224,3))

In [0]:
#predicting
accuracy_score(model.predict(train[:1,:,:,:]).numpy(),train_target[:1, 1:])

In [0]:
train_target[0:10, 1:]

In [0]:
Final_model.predict(feature[80:85,:,:,:]).cpu().numpy()

In [0]:
targets[80:85].numpy()

In [0]:
targets

In [0]:
_1_hidtrain.predict(data=test[0])

In [0]:
e

In [0]:
z

In [0]:
class train_model:
  
  def __init__(self,cnn = None,model = None, lr=0.01):
    self.cnn = cnn
    self.model = model
    self.criterion = None
    self.optimizer = None
    self.weights = None
    self.y_scores_train = []
    self.y_scores_test = []
    self.train_loss = []
    self.test_loss = []
    self.train_acc = []
    self.test_acc = []
    self.train_rec = []
    self.test_rec = []
    self.train_f1_scr = []
    self.test_f1_scr = []
    self.train_pre = []
    self.test_pre = []
    self.t_epch = 0
    self.loaded_tr = 0
    self.loaded_vl = 0
    
  def train(self,train_loader=None, valid_loader=None,criterion=None,optimizer=None,epochs=5,batch_size=[400,40],lst_val_scr=0):
    
    model = self.model
    cnn = self.cnn
    valid_rec_min = lst_val_scr
    train_on_gpu = True #tch.cuda.is_available()
    new_state = model.state_dict()
    self.t_epch += epochs

    if len(batch_size) == 2:
      batch_s_t = batch_size[0]
      batch_s_v = batch_size[1]
    else:
      batch_s_t = batch_size[0]
      batch_s_v = batch_size[0]    

    for epoch in range(epochs):
      # keep track of training and validation loss and scores
        train_loss    = 0.0
        test_loss    = 0.0
        train_acc     = 0.0
        train_pre     = 0.0
        train_rec     = 0.0
        train_f1_scr  = 0.0
        test_acc      = 0.0
        test_pre      = 0.0
        test_rec      = 0.0
        test_f1_scr   = 0.0
      
      ###################
      # train the model #
      ###################
        model.cuda()
        for data, target in train_loader:
            self.loaded_tr += len(data)
       
          # move tensors to GPU if CUDA is available
            if train_on_gpu: 
                cnn.eval()
                model.train()
                data, target = data.cuda(), target.cuda()
            else:
                cnn.eval()
                model.cpu()
                model.train()
                data, target = data.cpu(), target.cpu()
     
          # clear the gradients of all optimized variables
            optimizer.zero_grad()
     
            data = data.cpu()
            img = data[:,:,:,:3]
            img2 = data[:,:,:,3:]
            print(f'in Train-->({self.loaded_tr}) \n img1 --> {img.shape} \n img2 --> {img2.shape}')
            img = cnn(tch.from_numpy(img.numpy().transpose((0, 3, 1, 2))))
            img2 = cnn(tch.from_numpy(img2.numpy().transpose((0, 3, 1, 2))))

            data = tch.cat((img,img2),1).cuda()
          
          
          
          # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #print(f'output_train is {output} and target is {target}')
          # calculate the batch loss
            loss = criterion(output, target)                   
      
          # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
      
          # perform a single optimization step (parameter update)
            optimizer.step()
       
          # update training loss
            train_loss     +=  loss.item()*data.size(0)
            train_acc      +=  accuracy_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_pre      +=  precision_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_rec      +=  recall_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            train_f1_scr   +=  f1_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            self.save_model()
        self.loaded_tr = 0


      ######################    
      # validate the model #
      ######################
        cnn.eval()
        model.eval()
     
        for data, target in valid_loader:
            self.loaded_vl += len(data)
          # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
                data = data.cpu()
                img = data[:,:,:,:3]
                img2 = data[:,:,:,3:]
                print(f'in val-->({self.loaded_vl}) \n img1 --> {img.shape} \n img2 --> {img2.shape}')
                img = cnn(tch.from_numpy(img.numpy().transpose((0, 3, 1, 2))))
                img2 = cnn(tch.from_numpy(img2.numpy().transpose((0, 3, 1, 2))))

                data = tch.cat((img,img2),1).cuda()
                
          # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            #print(f'output_val is {output} and target is {target}')
          # calculate the batch loss
            loss = criterion(output, target)
                    
          # update average validation loss 
            test_loss     += loss.item()*data.size(0)
            test_acc       += accuracy_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_pre       += precision_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_rec       +=  recall_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            test_f1_scr    +=  f1_score(target.cpu()[:,1:], output.cpu().topk(1)[1])
            
        self.loaded_vl = 0       
      # calculate average losses and scores
        train_loss = train_loss/round(len(train_loader.dataset)/batch_size)
        test_loss = test_loss/round(len(valid_loader.dataset)/batch_size)
      
        train_acc       =   1 if  train_acc/round(len(train_loader.dataset)/batch_s_t) > 1 else train_acc/round(len(train_loader.dataset)/batch_s_t)

        train_pre       =   1 if train_pre/round(len(train_loader.dataset)/batch_s_t) > 1 else train_pre/round(len(train_loader.dataset)/batch_s_t)

        train_rec       =   1 if train_rec/round(len(train_loader.dataset)/batch_s_t) > 1 else train_rec/round(len(train_loader.dataset)/batch_s_t)
                                     
        train_f1_scr    =   1 if train_f1_scr/round(len(train_loader.dataset)/batch_s_t) > 1 else train_f1_scr/round(len(train_loader.dataset)/batch_s_t)

        test_acc        =   1 if test_acc/round(len(valid_loader.dataset)/batch_s_v) > 1 else test_acc/round(len(valid_loader.dataset)/batch_s_v)

        test_pre        =   1 if test_pre/round(len(valid_loader.dataset)/batch_s_v) > 1 else test_pre/round(len(valid_loader.dataset)/batch_s_v)
  
        test_rec        =   1 if test_rec/round(len(valid_loader.dataset)/batch_s_v,5) > 1 else test_rec/round(len(valid_loader.dataset)/batch_s_v,5)

        test_f1_scr    =   1 if test_f1_scr/round(len(valid_loader.dataset)/batch_s_v,5) > 1 else test_f1_scr/round(len(valid_loader.dataset)/batch_s_v,5) 

      

            
      #append train scores
        self.train_loss.append(train_loss)
        self.train_acc.append(train_acc)
        self.train_rec.append(train_rec)
        self.train_f1_scr.append(train_f1_scr)
        self.train_pre.append(train_pre)
        
      #append test scores
        self.test_loss.append(test_loss)
        self.test_acc.append(test_acc)
        self.test_rec.append(test_rec)
        self.test_f1_scr.append(test_f1_scr)
        self.test_pre.append(test_pre)
        
      # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} '.format(epoch, train_loss, test_loss))
      
        print('train Accuracy score : {:.6f} \ttrain Precision Score: {:.6f} \ttrain Recall Score: {:.6f}\ttrain f1 Score: {:.6f}'.format(train_acc, train_pre, train_rec, train_f1_scr))
      
        print('test Accuracy score : {:.6f} \ttest Precision Score: {:.6f} \ttest Recall Score: {:.6f}\ttest f1 Score: {:.6f}'.format(test_acc, test_pre, test_rec, test_f1_scr))
        self.model = model
        self.cnn = cnn
        self.save_model()
      # save model if validation loss has decreased
        if test_rec > valid_rec_min:
            print('test recall increased ({:.6f} --> {:.6f}).  Saving model ...\n\n'.format(valid_rec_min,test_rec))
            
            #Save State_dict
            fc_model_info = {'model_arc'  : model,
                             'state_dict' : model.state_dict()}
            
            cnn_model_info = {'model_arc'  : cnn,
                              'state_dict' : cnn.state_dict()}
            

            torch.save(fc_model_info, 'fcmodel_deploy0.pt')
            torch.save(cnn_model_info, 'cnnmodel_deploy0.pt')
            valid_rec_min = test_rec

  def predict(self,data):
    model = self.model
    model.cpu().eval()
    cnn = self.cnn
    
    cnn.cpu().eval()
    data = data.cpu()
    img = data[:,:,:,:3]
    img2 = data[:,:,:,3:]
      f
    img = cnn(tch.from_numpy(img.numpy().transpose((0, 3, 1, 2))))
    img2 = cnn(tch.from_numpy(img2.numpy().transpose((0, 3, 1, 2))))

    data = tch.cat((img,img2),1)

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    
    return output.cpu().topk(1)[1].detach().numpy()
  
  
  def save_model(self):
    model = self.model
    cnn = self.cnn

      #Save State_dict
    fc_model_info = {'model_arc'  : model,
                     'state_dict' : model.state_dict()}

    cnn_model_info = {'model_arc'  : cnn,
                      'state_dict' : cnn.state_dict()}

    torch.save(fc_model_info, 'fcmodel_gen0.pt')
    torch.save(cnn_model_info, 'cnnmodel_gen0.pt')

In [0]:
# modelk = models.resnet152(pretrained=True)

In [0]:
# modelg.fc = nn.Identity()

In [0]:
# #Save State_dict
# model_info = {'model_arc'  : modelg,
#              'state_dict' : modelg.state_dict()}

# torch.save(model_info, 'trmodel.pt')

In [0]:
# Load Saved model arch
vgg = torch.load('trmodel.pt')
model = vgg['model_arc']
model.load_state_dict(vgg['state_dict'])
for param in model.parameters():
  param.requires_grad = False
  
  
imgt = model(img)
img2t = model(img2)
tch.cat((imgt,img2t),1).shape

In [0]:
#Building the model
# neural network architecture
fc = nn.Sequential(OrderedDict([ 
                                   ("input", nn.Linear(4096, 1000)),
                                   ("act_1", nn.ReLU()),
                                   ("hidd", nn.Linear(1000, 2)),
                                   ("out", nn.Softmax(dim=1))
                               ]))
mod = tch.load('fcmodel_gen0.pt')
fc = mod['model_arc']
fc.load_state_dict(mod['state_dict'])

resnet = torch.load('cnnmodel_gen0.pt')
cnn = resnet['model_arc']
cnn.load_state_dict(resnet['state_dict'])

# for param in cnn.parameters():
#   param.requires_grad = True
  
# cnn = models.resnet50(pretrained=True)
ct = 0
for child in cnn.children():
  ct += 1
  if ct < 8:
      for param in child.parameters():
          param.requires_grad = False

